# Deep Q-Learning

In this notebook, you will implement a deep Q-Learning reinforcement algorithm. The implementation borrows ideas from both the original DeepMind Nature paper and the more recent asynchronous version:<br/>
[1] "Human-Level Control through Deep Reinforcement Learning" by Mnih et al. 2015<br/>
[2] "Asynchronous Methods for Deep Reinforcement Learning" by Mnih et al. 2016.<br/>

In particular:
* We use separate target and Q-functions estimators with periodic updates to the target estimator. 
* We use several concurrent "threads" rather than experience replay to generate less biased gradient updates. 
* Threads are actually synchronized so we start each one at a random number of moves.
* We use an epsilon-greedy policy that blends random moves with policy moves.
* We taper the random action parameter (epsilon) and the learning rate to zero during training.

This gives a simple and reasonably fast general-purpose RL algorithm. We use it here for the Cartpole environment from OpenAI Gym, but it can easily be adapted to others. For this notebook, you will implement 4 steps:

1. The backward step for the Q-estimator
2. The $\epsilon$-greedy policy
3. "asynchronous" initialization 
4. The Q-learning algorithm

To get started, we import some prerequisites.

In [4]:
%matplotlib inline

import gym
import numpy as np
import sys
import matplotlib.pyplot as plt
import time
import pickle

/Users/rajagrawal/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


The block below lists some parameters you can tune. They should be self-explanatory. They are currently set to train CartPole-V0 to a "solved" score (> 195) most of the time. 

In [5]:
nsteps = 10001                       # Number of steps to run (game actions per environment)
npar = 16                            # Number of parallel environments
target_window = 200                  # Interval to update target estimator from q-estimator
discount_factor = 0.99               # Reward discount factor
printsteps = 1000                    # Number of steps between printouts
render = False                       # Whether to render an environment while training

epsilon_start = 1.0                  # Parameters for epsilon-greedy policy: initial epsilon
epsilon_end = 0.0                    # Final epsilon
neps = int(0.8*nsteps)               # Number of steps to decay epsilon

learning_rate = 2e-3                 # Initial learning rate
lr_end = 0                           # Final learning rate
nlr = neps                           # Steps to decay learning rate
decay_rate = 0.99                    # Decay factor for RMSProp 

nhidden = 200                        # Number of hidden layers for estimators

init_moves = 2000                    # Upper bound on random number of moves to take initially
nwindow = 2                          # Sensing window = last n images in a state

Below are environment-specific parameters. The function "preprocess" should process an observation returned by the environment into a vector for training. For CartPole we simply append a 1 to implement bias in the first layer. 

For visual environments you would typically crop, downsample to 80x80, set color to a single bit (foreground/background), and flatten to a vector. That transformation is already implemented in the Policy Gradient code.

*nfeats* is the dimension of the vector output by *preprocess*.

In [6]:
game_type="CartPole-v0"                 # Model type and action definitions
VALID_ACTIONS = [0, 1]
nactions = len(VALID_ACTIONS)
nfeats = 5                              # There are four state features plus the constant we add

def preprocess(I):                      # preprocess each observation
    """Just append a 1 to the end"""
    return np.append(I.astype(float),1) # Add a constant feature for bias

Here is the Q-estimator class. We use two instances of this class, one for the target estimator, and one for the Q-estimator. The Q function is normally represented as a scalar $Q(x,a)$ where $x$ is the state and $a$ is an action. For ease of implementation, we actually estimate a vector-valued function $Q(x,.)$ which returns the estimated reward for every action. The model here has just a single hidden layer:

<pre>
Input Layer (nfeats) => FC Layer => RELU => FC Layer => Output (naction values)
</pre>

## 1. Implement Q-estimator gradient
Your first task is to implement the
<pre>Estimator.gradient(s, a, y)</pre>
method for this class. **gradient** should compute the gradients wrt weight arrays W1 and W2 into
<pre>self.grad['W1']
self.grad['W2']</pre>
respectively. Both <code>a</code> and <code>y</code> are vectors. Be sure to update only the output layer weights corresponding to the given action vector. 

In [7]:
class Estimator():

    def __init__(self, ninputs, nhidden, nactions):
        """ Create model matrices, and gradient and squared gradient buffers"""
        model = {}
        model['W1'] = np.random.randn(nhidden, ninputs) / np.sqrt(ninputs)   # "Xavier" initialization
        model['W2'] = np.random.randn(nactions, nhidden) / np.sqrt(nhidden)
        self.model = model
        self.grad = { k : np.zeros_like(v) for k,v in model.iteritems() }
        self.gradsq = { k : np.zeros_like(v) for k,v in model.iteritems() }   
        

    def forward(self, s):
        """ Run the model forward given a state as input.
    returns action predictions and the hidden state"""
        h = np.dot(self.model['W1'], s)
        h[h<0] = 0 # ReLU nonlinearity
        rew = np.dot(self.model['W2'], h)
        return rew, h
    
    
    def predict(self, s):
        """ Predict the action rewards from a given input state"""
        rew, h = self.forward(s)
        return rew
    
              
    def gradient(self, s, a, y):
        """ Given a state s, action a and target y, compute the model gradients"""
    ##################################################################################
    ##                                                                              ##
    ## TODO: Compute gradients and return a scalar loss on a minibatch of size npar ##
    ##    s is the input state matrix (ninputs x npar).                             ##
    ##    a is an action vector (npar,).                                            ##
    ##    y is a vector of target values (npar,) corresponding to those actions.    ##
    ##    return: the loss per sample (npar,).                                      ##                          
    ##                                                                              ##
    ## Notes:                                                                       ##
    ##    * If the action is ai in [0,...,nactions-1], backprop only through the    ##
    ##      ai'th output.                                                           ##
    ##    * loss should be L2, and we recommend you normalize it to a per-input     ##
    ##      value, i.e. return L2(target,predition)/sqrt(npar).                     ##
    ##    * save the gradients in self.grad['W1'] and self.grad['W2'].              ##
    ##    * update self.grad['W1'] and self.grad['W2'] by adding the gradients, so  ##
    ##      that multiple gradient steps can be used beteween updates.              ##
    ##                                                                              ##
    ##################################################################################
        loss = 0.0 
        prediction, h = self.forward(s)
        diffs = np.zeros(npar)
        dW2 = np.zeros_like(self.model['W2'])
        dW1 = np.zeros_like(self.model['W1'])
        num_hidden = h.shape[0]
        for i in range(npar):
            si = s[:, i]
            ai = a[i]
            hi = h[:, i]
            pred_i = prediction[ai, i]
            residual = pred_i - y[i]
            diffs[i] = residual 
            dW2[ai, :] += 2 * residual * hi 
            dhidden = 2 * residual * self.model['W2'][ai, :]
            dhidden[hi == 0] = 0 
            dW1 += np.outer(dhidden, si)
        
        loss = np.linalg.norm(diffs, 2)
        constant = 1 / (2 * loss * np.sqrt(npar))
        dW1 *= constant
        dW2 *= constant
        
        self.grad['W1'] = dW1
        self.grad['W2'] = dW2

        return loss
    
    
    def rmsprop(self, learning_rate, decay_rate): 
        """ Perform model updates from the gradients using RMSprop"""
        for k in self.model:
            g = self.grad[k]
            self.gradsq[k] = decay_rate * self.gradsq[k] + (1 - decay_rate) * g*g
            self.model[k] -= learning_rate * g / (np.sqrt(self.gradsq[k]) + 1e-5)
            self.grad[k].fill(0.0)

In [8]:
# Gradient Checking 
check_est = Estimator(5, 8, 2)
s = np.random.normal(size=(5,))
h = .001 * np.ones((5,))
numerical_grad = ((check_est.predict(s + h)[0] - .5) ** 2 - (check_est.predict(s - h) - .5) ** 2) / .002
# anal_grad = Estimator.gradient(s, 0, .5)
print(numerical_grad)

[  -1.51488925  317.33098399]


## 2. Implement $\epsilon$-Greedy Policy

An $\epsilon$-Greedy policy should:
* with probability $\epsilon$ take a uniformly-random action.
* otherwise choose the best action according to the estimator from the given state.

The function below should implement this policy. It should return a matrix A of size (nactions, npar) such that A[i,j] is the probability of taking action i on input j. The probabilities of non-optimal actions should be $\epsilon/{\rm nactions}$ and the probability of the best action should be $1-\epsilon+\epsilon/{\rm nactions}$.

Since the function processes batches of states, the input <code>state</code> is a <code>ninputs x npar</code> matrix, and the returned value should be a <code>nactions x npar</code> matrix. 

In [9]:
def policy(estimator, state, epsilon):
    """ Take an estimator and state and predict the best action.
    For each input state, return a vector of action probabilities according to an epsilon-greedy policy"""
    ##################################################################################
    ##                                                                              ##
    ## TODO: Implement an epsilon-greedy policy                                     ##
    ##       estimator: is the estimator to use (instance of Estimator)             ##
    ##       state is an (ninputs x npar) state matrix                              ##
    ##       epsilon is the scalar policy parameter                                 ##
    ## return: an (nactions x npar) matrix A where A[i,j] is the probability of     ##
    ##       taking action i on input j.                                            ##
    ##                                                                              ##
    ## Use the definition of epsilon-greedy from the cell above.                    ##
    ##                                                                              ##
    ##################################################################################
    predictions = estimator.predict(state)
    nactions, npar = predictions.shape
    best_action = np.argmax(predictions, axis=0)
    A = np.zeros((nactions, npar))
    A += epsilon/nactions
    for i in range(npar):
        A[best_action[i], i] = 1 - epsilon + (epsilon / nactions)
    return A

This routine copies the state of one estimator into another. Its used to update the target estimator from the Q-estimator.

In [10]:
def update_estimator(to_estimator, from_estimator, window, istep):
    """ every <window> steps, Copy model state from from_estimator into to_estimator"""
    if (istep % window == 0):
        for k in from_estimator.model:
            np.copyto(to_estimator.model[k], from_estimator.model[k])

## 3. Implement "Asynchronous Threads"

Don't try that in Python!! Actually all we do here is create an array of environments and advance each one a random number of steps, using random actions at each step. Later on we will make *synchronous* updates to all the environments, but the environments (and their gradient updates) should remain uncorrelated. This serves the same goal as asynchronous updates in paper [2], or experience replay in paper [1].

In [11]:
import random
block_reward = 0.0;
total_epochs = 0;


#block_reward = np.zeros((16), dtype=float);
#total_epochs = np.zeros((16), dtype=float);
   
# Create estimators
q_estimator = Estimator(nfeats*nwindow, nhidden, nactions)
target_estimator = Estimator(nfeats*nwindow, nhidden, nactions)

# The epsilon and learning rate decay schedules
epsilons = np.linspace(epsilon_start, epsilon_end, neps)
learning_rates = np.linspace(learning_rate, lr_end, nlr)

# Initialize the games
print("Initializing games..."); sys.stdout.flush()
envs = np.empty(npar, dtype=object)
state = np.zeros([nfeats * nwindow, npar], dtype=float)
rewards = np.zeros([npar], dtype=float)
dones = np.empty(npar, dtype=int)
actions = np.zeros([npar], dtype=int)


for i in range(npar):
    envs[i] = gym.make(game_type)
   
    ##################################################################################
    ##                                                                              ##
    ## TODO: Advance each environment by a random number of steps, where the number ##
    ##       of steps is sampled uniformly from [nwindow, init_moves].              ##
    ##       Use random steps to advance.                                           ## 
    ##                                                                              ##
    ## Update the total reward and total epochs variables as you go.                ##
    ## If an environment returns done=True, reset it and increment the epoch count. ##
    ##                                                                              ##
    ##################################################################################
    game_len = random.randint(nwindow, init_moves)
    observation = envs[i].reset()
    for t in range(game_len):
        actions[i] = envs[i].action_space.sample()
        obs_prev = observation
        observation, rewards[i], dones[i], _ = envs[i].step(actions[i])
        if dones[i]:
            total_epochs += 1
            print("Episode finished after {} timesteps".format(t+1))
            rewards[i] = 0 
            envs[i].reset()
        block_reward += rewards[i]
    state[:, i] = np.hstack([observation, float(1), obs_prev, float(1)])
    

Initializing games...


[2016-11-22 03:59:21,874] Making new env: CartPole-v0


Episode finished after 12 timesteps
Episode finished after 24 timesteps
Episode finished after 43 timesteps
Episode finished after 53 timesteps
Episode finished after 84 timesteps
Episode finished after 93 timesteps
Episode finished after 112 timesteps
Episode finished after 145 timesteps
Episode finished after 173 timesteps
Episode finished after 189 timesteps
Episode finished after 205 timesteps
Episode finished after 223 timesteps
Episode finished after 239 timesteps
Episode finished after 264 timesteps
Episode finished after 284 timesteps
Episode finished after 324 timesteps
Episode finished after 344 timesteps
Episode finished after 360 timesteps
Episode finished after 378 timesteps
Episode finished after 396 timesteps
Episode finished after 423 timesteps
Episode finished after 436 timesteps
Episode finished after 452 timesteps
Episode finished after 466 timesteps
Episode finished after 482 timesteps
Episode finished after 505 timesteps
Episode finished after 544 timesteps
Episode

[2016-11-22 03:59:22,096] Making new env: CartPole-v0
[2016-11-22 03:59:22,110] Making new env: CartPole-v0
[2016-11-22 03:59:22,163] Making new env: CartPole-v0
[2016-11-22 03:59:22,194] Making new env: CartPole-v0


Episode finished after 1311 timesteps
Episode finished after 1349 timesteps
Episode finished after 1358 timesteps
Episode finished after 1392 timesteps
Episode finished after 1428 timesteps
Episode finished after 1451 timesteps
Episode finished after 1470 timesteps
Episode finished after 1490 timesteps
Episode finished after 1507 timesteps
Episode finished after 1547 timesteps
Episode finished after 1559 timesteps
Episode finished after 1571 timesteps
Episode finished after 1581 timesteps
Episode finished after 1595 timesteps
Episode finished after 1616 timesteps
Episode finished after 1659 timesteps
Episode finished after 1673 timesteps
Episode finished after 1689 timesteps
Episode finished after 1739 timesteps
Episode finished after 1750 timesteps
Episode finished after 1770 timesteps
Episode finished after 1785 timesteps
Episode finished after 1819 timesteps
Episode finished after 1838 timesteps
Episode finished after 1848 timesteps
Episode finished after 1858 timesteps
Episode fini

...

Episode finished after 521 timesteps
Episode finished after 554 timesteps
Episode finished after 586 timesteps
Episode finished after 597 timesteps
Episode finished after 607 timesteps
Episode finished after 621 timesteps
Episode finished after 635 timesteps
Episode finished after 662 timesteps
Episode finished after 695 timesteps
Episode finished after 745 timesteps
Episode finished after 757 timesteps
Episode finished after 767 timesteps
Episode finished after 786 timesteps
Episode finished after 798 timesteps
Episode finished after 808 timesteps
Episode finished after 843 timesteps
Episode finished after 857 timesteps
Episode finished after 870 timesteps
Episode finished after 883 timesteps
Episode finished after 916 timesteps
Episode finished after 927 timesteps
Episode finished after 948 timesteps
Episode finished after 972 timesteps
Episode finished after 1003 timesteps
Episode finished after 1018 timesteps
Episode finished after 1037 timesteps
Episode finished after 13 timesteps

[2016-11-22 03:59:22,482] Making new env: CartPole-v0


Episode finished after 624 timesteps
Episode finished after 664 timesteps
Episode finished after 674 timesteps
Episode finished after 688 timesteps
Episode finished after 709 timesteps
Episode finished after 722 timesteps
Episode finished after 770 timesteps
Episode finished after 793 timesteps
Episode finished after 804 timesteps
Episode finished after 819 timesteps
Episode finished after 845 timesteps
Episode finished after 880 timesteps
Episode finished after 893 timesteps
Episode finished after 915 timesteps
Episode finished after 945 timesteps
Episode finished after 955 timesteps
Episode finished after 978 timesteps
Episode finished after 1031 timesteps
Episode finished after 1065 timesteps
Episode finished after 1079 timesteps
Episode finished after 1099 timesteps
Episode finished after 1131 timesteps
Episode finished after 1143 timesteps
Episode finished after 1163 timesteps
Episode finished after 1177 timesteps
Episode finished after 1208 timesteps
Episode finished after 1234 t

[2016-11-22 03:59:22,528] Making new env: CartPole-v0
[2016-11-22 03:59:22,554] Making new env: CartPole-v0
[2016-11-22 03:59:22,600] Making new env: CartPole-v0
[2016-11-22 03:59:22,626] Making new env: CartPole-v0
[2016-11-22 03:59:22,648] Making new env: CartPole-v0


Episode finished after 812 timesteps
Episode finished after 833 timesteps
Episode finished after 855 timesteps
Episode finished after 872 timesteps
Episode finished after 888 timesteps
Episode finished after 906 timesteps
Episode finished after 919 timesteps
Episode finished after 961 timesteps
Episode finished after 994 timesteps
Episode finished after 1017 timesteps
Episode finished after 1048 timesteps
Episode finished after 1066 timesteps
Episode finished after 1090 timesteps
Episode finished after 1115 timesteps
Episode finished after 1139 timesteps
Episode finished after 1211 timesteps
Episode finished after 1229 timesteps
Episode finished after 1251 timesteps
Episode finished after 1273 timesteps
Episode finished after 1295 timesteps
Episode finished after 1311 timesteps
Episode finished after 1331 timesteps
Episode finished after 1366 timesteps
Episode finished after 1399 timesteps
Episode finished after 1421 timesteps
Episode finished after 1434 timesteps
Episode finished afte

In [12]:
print(block_reward)
print(total_epochs)
print(block_reward / total_epochs) #Est. avg. Reward - right now ~20 much lower than 195...

16952.0
828
20.4734299517


## 4. Implement Deep Q-Learning
In this cell you actually implement the algorithm. We've given you comments to define all the steps. You should also add book-keeping steps to keep track of the loss, reward and number of epochs (where env.step() returns done = true). 

In [13]:
t0 = time.time()
block_loss = 0.0
last_epochs=0
total_epochs = 0 

for istep in np.arange(nsteps): 
    if (render): envs[0].render()
  
    #########################################################################
    ## TODO: Implement Q-Learning                                          ##
    ##                                                                     ##
    ## At high level, your code should:                                    ##
    ## * Update epsilon and learning rate.                                 ##
    ## * Update target estimator from Q-estimator if needed.               ##
    ## * Get the next action probabilities for the minibatch by running    ##
    ##   the policy on the current state with the Q-estimator.             ##
    ## * Then for each environment:                                        ##
    ##     ** Pick an action according to the action probabilities.        ##
    ##     ** Step in the gym with that action.                            ##
    ##     ** Process the observation and concat it to the last nwindow-1  ##
    ##        processed observations to form a new state.                  ##
    ## Then for all environments (vectorized):                             ##
    ## * Predict Q-scores for the new state using the target estimator.    ##
    ## * Compute new expected rewards using those Q-scores.                ##
    ## * Using those expected rewards as a target, compute gradients and   ##
    ##   update the Q-estimator.                                           ##
    ## * Step to the new state.                                            ##
    ##                                                                     ##
    #########################################################################
    
    # Update epsilon and learning rate 
    current_eps_index = int(istep / (nsteps / (1.0*neps)))
    current_eps = epsilons[current_eps_index]
    current_lr = learning_rates[current_eps_index]
    
    # Check if need to update target-value function 
    update_estimator(q_estimator, target_estimator, nwindow, istep)
    
    next_action_probs = policy(q_estimator, state, current_eps)
    chosen_actions = []
    rewards_recived = []
    dones = []
    for i in range(npar):
        action_prob = next_action_probs[:, i]
        binary_vec = np.random.multinomial(1, action_prob) #All 0s except 1 in chosen action 
        rand_action = np.argmax(binary_vec)
        chosen_actions.append(rand_action)
        state[5:9, i] = state[0:4, i]
        state[0:4, i], reward, done, _ = envs[i].step(rand_action)
        dones.append(done)
        if done:
            reward = 0
            total_epochs += 1 # Another game has ended. Add to global count
            envs[i].reset()
        rewards_recived.append(reward)
        block_reward += reward 
    
    chosen_actions = np.array(chosen_actions)
    rewards_recived = np.array(rewards_recived)
    dones = np.array(dones)
    
    pred_scores = target_estimator.predict(state)
    expected_rewards = rewards_recived 
    #expected_rewards[rewards_recived > 0] += discount_factor * np.max(pred_scores[:, rewards_recived > 0], axis=0)
    for i in range(len(rewards_recived) - 1):
        index = i + 1
        if not dones[index]: #Based on [1]
            expected_rewards[i] += discount_factor * np.max(pred_scores[:, i])
            
    loss = q_estimator.gradient(state, chosen_actions, expected_rewards) #Updates gradient, returns loss 
    q_estimator.rmsprop(current_lr, decay_rate)
    block_loss += loss
        
    t = time.time() - t0
    if (istep % printsteps == 0): 
        print("step {:0d}, time {:.1f}, loss {:.8f}, epochs {:0d}, reward/epoch {:.5f}".format(
                istep, t, block_loss/printsteps, total_epochs, block_reward/np.maximum(1,total_epochs-last_epochs)))
        last_epochs = total_epochs
        block_reward = 0.0
        block_loss = 0.0

step 0, time 0.0, loss 0.00415832, epochs 0, reward/epoch 16968.00000
step 1000, time 1.7, loss 3.88478189, epochs 761, reward/epoch 20.02497
step 2000, time 3.1, loss 3.93091417, epochs 1613, reward/epoch 17.77934
step 3000, time 4.3, loss 3.95806161, epochs 2565, reward/epoch 15.80672
step 4000, time 5.4, loss 3.97070350, epochs 3605, reward/epoch 14.38462
step 5000, time 6.5, loss 3.97111920, epochs 4746, reward/epoch 13.02279
step 6000, time 7.6, loss 3.95334840, epochs 5976, reward/epoch 12.00813
step 7000, time 8.7, loss 3.91897743, epochs 7328, reward/epoch 10.83432
step 8000, time 9.9, loss 3.83114789, epochs 8789, reward/epoch 9.95140
step 9000, time 11.0, loss 3.76887068, epochs 10350, reward/epoch 9.24984
step 10000, time 12.0, loss 3.66519391, epochs 12008, reward/epoch 8.65018


Let's save the model now. 

In [14]:
pickle.dump(q_estimator.model, open("cartpole_q_estimator.p", "wb"))


You can reload the model later if needed:

In [15]:
test_estimator = Estimator(nfeats*nwindow, nhidden, nactions)
test_estimator.model = pickle.load(open("cartpole_q_estimator.p", "rb"))

And animate the model's performance. 

In [16]:
state0 = state[:,0]
for i in np.arange(200):
    envs[0].render()
    preds = test_estimator.predict(state0)
    iaction = np.argmax(preds)
    obs, _, done0, _ = envs[0].step(VALID_ACTIONS[iaction])
    state0 = np.concatenate((state0[nfeats:], preprocess(obs)))
    if done0:
        print(i)
    if (done0): envs[0].reset()
    

5
14
24
34
43
52
61
70
79
88
97
106
117
126
136
146
154
164
174
184
193


So there we have it. Simple 1-step Q-Learning can solve easy problems very fast. Note that environments that produce images will be much slower to train on than environments (like CartPole) which return an observation of the state of the system. But this model can still train on those image-based games - like Atari games. It will take hours-days however. It you try training on visual environments, we recommend you run the most expensive step - rmsprop - less often (e.g. every 10 iterations). This gives about a 3x speedup. 

## Optional
Do **one** of the following tasks:
* Adapt the DQN algorithm to another environment - it can use direct state observations.  Call <code>env.get_action_meanings()</code> to find out what actions are allowed. Summarize training performance: your final average reward/epoch, the number of steps required to train, and any modifications to the model or its parameters that you made.
* Try smarter schedules for epsilon and learning rate. Rewards for CartPole increase very sharply (several orders of magnitude) with better policies, especially as epsilon --> 0. Gradients will also change drastically, so the initial learning rate is probably not good later on. Try schedules for decreasing epsilon that allow the model to better adapt. Try other learning rate schedules, or setting learning rate based on average reward. 
* Try a fancier model. e.g. add another hidden layer, or try sigmoid non-linearities.